In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
sample = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')

In [ ]:
labels.head()

In [ ]:
sample.head()

In [ ]:
def jpg(id):
    return id+".jpg"

In [ ]:
labels["id"] = labels["id"].apply(jpg)
sample["id"] = sample["id"].apply(jpg)

In [ ]:
sample.count()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator as Imgen
from keras.preprocessing import image

In [ ]:
from tensorflow import keras

In [ ]:
datagenerated = Imgen(preprocessing_function = keras.applications.nasnet.preprocess_input,
                      shear_range = 0.2,
                     horizontal_flip = True,
                     validation_split = 0.1)

In [ ]:
train = datagenerated.flow_from_dataframe(
labels,
directory = '../input/dog-breed-identification/train',
x_col = 'id',
y_col = 'breed',
subset = 'training',
color_mode = 'rgb',
class_mode = 'categorical',
target_size = (331,331),
batch_size = 32,
shuffle =True,
seed = 123)

val_ds = datagenerated.flow_from_dataframe(
labels,
directory = '../input/dog-breed-identification/train',
x_col = 'id',
y_col = 'breed',
subset = 'validation',
color_mode = 'rgb',
class_mode = 'categorical',
target_size = (331,331),
batch_size = 32,
shuffle =True,
seed = 123)


In [ ]:
names = train.class_indices

In [ ]:
class_names = [k for k,v in names.items()]

In [ ]:
class_names[:10]

In [ ]:
x,y = next(train)
print(y.shape)
print(x.shape)

In [ ]:
y[0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot(images,labels):
    plt.figure(figsize = [20,15])
    for i in range(30):
        plt.subplot(5,6,i+1)
        plt.imshow(images[i])
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis('off')

In [ ]:
plot(x,y)

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [ ]:
transfer_model = InceptionResNetV2(include_top = False,
                                   weights ='imagenet',
                                  input_shape = (331,331,3)
                                  )
transfer_model.trainable = False

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,MaxPooling2D,Dropout,Conv2D,GlobalAveragePooling2D,Flatten

In [ ]:
model = Sequential()
model.add(transfer_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(120,activation = 'softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
callback = [keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=2),
           keras.callbacks.ModelCheckpoint("MyModel.h5",save_best_only =True,verbose =2)]

In [ ]:
history = model.fit(train,epochs = 25,validation_data = val_ds,callbacks = callback)

In [ ]:
plt.figure(figsize=(7,6))
plt.plot(history.epoch,history.history['accuracy'],label ="training_acc")
plt.plot(history.epoch,history.history['val_accuracy'],label ="validation_acc")
plt.title("Accuracy")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.epoch,history.history["loss"],label = "training_loss")
plt.plot(history.epoch,history.history["val_loss"],label = "validation_loss")
plt.title("Loss")
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model
model = load_model("./MyModel.h5")

In [ ]:
testgen = Imgen(preprocessing_function=keras.applications.inception_resnet_v2.preprocess_input)

In [ ]:
test = testgen.flow_from_dataframe(
sample,
directory='/kaggle/input/dog-breed-identification/test',
    x_col = 'id',
    y_col = None,
    color_mode = 'rgb',
    target_size = (331,331),
    class_mode = None,
    batch_size = 32,
    shuffle = False
    
)

In [ ]:
pred = model.predict(test,verbose =2)

In [ ]:
submission = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')
submission.head()

In [ ]:
submission.iloc[:,1:] =pred

In [ ]:
submission.head()

In [ ]:
final_sub = submission.set_index('id')
final_sub.to_csv('Submission.csv')